In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/lepidoptera"

early_regex = r"^.*-(early)$"
unidentified_regex = r"^.*-(spp|genera|genera-spp)$"
early_or_unidentified_regex = r"^.*-(early|spp|genera|genera-spp)$"

In [5]:
import json

input_data = {
    "insect-dataset/src/class_details.ifoundbutterflies.org.json": "Butterfly",
    "insect-dataset/src/class_details.mothsofindia.org.json": "Moth",
}
combined_data = {}

for src, name in input_data.items():
    with open(src, "r", encoding="utf-8") as file:
        data = json.load(file)
    for key, value in data.items():
        if 'name' in value and not re.match(r"(?i)^.*(moth|moths|butterfly|butterflies|fly|flies)$", value['name']):
            value['name'] += " " + name
        combined_data[key] = value

for species_type in ['moth', 'butterfly']:
    for class_name in os.listdir(f"{dataset_dir}/../{species_type}/data"):
        if not re.match(early_or_unidentified_regex, class_name) and (class_name not in combined_data or 'name' not in combined_data[class_name]):
            if class_name not in combined_data:
                combined_data[class_name] = {}
            name = re.sub(r"-", " ", class_name, count=1).title() + " " + species_type.title()
            combined_data[class_name]['name'] = name

with open("models/class_details.lepidoptera.json", "w", encoding="utf-8") as file:
    json.dump(combined_data, file, indent=4)

In [2]:
shutil.copy("insect-dataset/src/class_details.indianodonata.org.json", "models/class_details.odonata.json")

'models/class_details.odonata.json'

In [3]:
shutil.copy("insect-dataset/src/class_details.indiancicadas.org.json", "models/class_details.cicada.json")

'models/class_details.cicada.json'

In [8]:
classes = [class_name for class_name in os.listdir(f"insect-dataset/src/mothsodindia.org")]
for class_name in os.listdir(f"insect-dataset/src/cicada.inaturalist.org"):
    if class_name in classes or re.sub(r"-early$", "", class_name) in classes:
        shutil.move(f"insect-dataset/src/cicada.inaturalist.org/{class_name}", f"insect-dataset/src/odonata.inaturalist.org/{class_name}")